In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import skimage
from skimage.io import imread, imshow
from skimage.transform import resize

In [3]:
def readImgs(path):
    imgs = []
    i = 0
    for filename in os.listdir(path): 
        i += 1
        if i == 251:
            break
        if filename.endswith('.jpg'):
            img = imread(os.path.join(path,filename))
            resized = resize(img, (256, 256), anti_aliasing=True)
            imgs.append(resized)
    return imgs

In [4]:
train_df_path = 'deepfake_database/deepfake_database/train:test/df'
train_real_path = 'deepfake_database/deepfake_database/train:test/real'
deepfakes_train = readImgs(train_df_path)
real_train = readImgs(train_real_path)

In [5]:
print(len(deepfakes_train))
print(len(real_train))

250
250


In [6]:
y_train = [0]*250 + [1]*250
X_train = deepfakes_train + real_train

In [7]:
y_train = np.array(y_train)
X_train = np.array(X_train)

In [8]:
X_train.shape

(500, 256, 256, 3)

# Level One: Inception Resnet

In [14]:
from keras.applications import InceptionResNetV2
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
inception = InceptionResNetV2(include_top=False, weights=None, input_shape=(256, 256, 3))
inception.trainable = True
model = Sequential()
model.add(inception)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='Adam')

In [15]:
model.fit(X_train, y_train, batch_size = 50, validation_split = 0.2, epochs = 5, verbose = 1)

Train on 400 samples, validate on 100 samples
Epoch 1/5
400/400 [==============================] - 836s 2s/sample - loss: 1.0445 - val_loss: 0.8394
Epoch 2/5
400/400 [==============================] - 749s 2s/sample - loss: 0.5040 - val_loss: 0.9369
Epoch 3/5
400/400 [==============================] - 807s 2s/sample - loss: 0.4053 - val_loss: 1.0898
Epoch 4/5
400/400 [==============================] - 722s 2s/sample - loss: 0.4116 - val_loss: 0.3785
Epoch 5/5
400/400 [==============================] - 691s 2s/sample - loss: 0.3389 - val_loss: 0.3345


In [33]:
print("The order of the files are (df is DeepFake):")
for name in image_names:
        print(name)

prob_real_dev = model.predict(X_test)
actual_pred_dev = convertToActual(prob_real_dev)
print('Predicted probability of the image being real Developer:', prob_real_dev,'\nPredicted class :', actual_pred_dev)
print('Deepfake detection accuracy is:', computeAccuracy(actual_pred_dev, y_test), '%')

The order of the files are (df is DeepFake):
df/elon_fake_1.png
df/elon_fake_2.png
df/obama_fake.png
real/elon_real.png
real/obama_real.png
Predicted probability of the image being real Developer: [[0.70221025]
 [0.7025099 ]
 [0.7004069 ]
 [0.7137978 ]
 [0.72679734]] 
Predicted class : [[1.]
 [1.]
 [1.]
 [1.]
 [1.]]
Deepfake detection accuracy is: 40.0 %


# Level Two: MesoNet

In [10]:
import tensorflow as tf
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU

Using TensorFlow backend.


In [46]:
def init_model(img_width): 
    x = Input(shape = (img_width, img_width, 3))
    x1 = Conv2D(8, (3, 3), padding='same', activation = 'relu')(x)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
        
    x2 = Conv2D(8, (5, 5), padding='same', activation = 'relu')(x1)
    x2 = BatchNormalization()(x2)
    x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)
        
    x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
    x3 = BatchNormalization()(x3)
    x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
    x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
    x4 = BatchNormalization()(x4)
    x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
        
    y = Flatten()(x4)
    y = Dropout(0.5)(y)
    y = Dense(16)(y)
    y = LeakyReLU(alpha=0.1)(y)
    y = Dropout(0.5)(y)
    y = Dense(1, activation = 'sigmoid')(y)
    return Model(inputs = x, outputs = y)

In [47]:
Meso4 = init_model(256)
opt = Adam(lr = 0.001)
Meso4.compile(optimizer = opt, loss = 'binary_crossentropy')

In [48]:
Meso4.fit(X_train, y_train, batch_size = 50, validation_split = 0.2, epochs = 5)

Train on 400 samples, validate on 100 samples
Epoch 1/5
400/400 [==============================] - 30s 76ms/step - loss: 2.0292 - val_loss: 0.7274
Epoch 2/5
400/400 [==============================] - 21s 52ms/step - loss: 1.1109 - val_loss: 0.6889
Epoch 3/5
400/400 [==============================] - 20s 49ms/step - loss: 0.8715 - val_loss: 0.7026
Epoch 4/5
400/400 [==============================] - 23s 57ms/step - loss: 0.7349 - val_loss: 0.7127
Epoch 5/5
400/400 [==============================] - 24s 59ms/step - loss: 0.6808 - val_loss: 0.7215


In [38]:
print("The order of the files are (df is DeepFake):")
for name in image_names:
        print(name)
Meso4 = load_model('Meso4.h5')
prob_real_meso = Meso4.predict(X_test)

actual_pred_Meso = convertToActual(prob_real_meso)
print('Predicted probability of the image being real Meso:', prob_real_meso,'\nPredicted class :', actual_pred_Meso)
print('Deepfake detection accuracy is:', computeAccuracy(actual_pred_Meso, y_test), '%')

The order of the files are (df is DeepFake):
df/elon_fake_1.png
df/elon_fake_2.png
df/obama_fake.png
real/elon_real.png
real/obama_real.png
Predicted probability of the image being real Meso: [[0.4955473 ]
 [0.51936394]
 [0.55302715]
 [0.5075346 ]
 [0.49840945]] 
Predicted class : [[0.]
 [1.]
 [1.]
 [1.]
 [0.]]
Deepfake detection accuracy is: 40.0 %


In [50]:
Meso4.save('Meso4.h5')

# Level 3: Xception

In [7]:
import tensorflow as tf
from keras.layers import Dense
from keras import Model
from keras.applications.xception import Xception

conv_base = Xception(include_top = False, weights = None, pooling = 'avg', input_shape = (256, 256, 3))

output_layer =  Dense(activation = 'sigmoid')(conv_base.output)

modified_classifier = Model(conv_base.input, output_layer)

modified_classifier.compile(optimizer = 'Adam', loss = 'binary_crossentropy')


Using TensorFlow backend.


In [8]:
modified_classifier.fit(X_train, y_train, batch_size = 50, validation_split = 0.2, epochs = 5)

Train on 400 samples, validate on 100 samples
Epoch 1/5
400/400 [==============================] - 1304s 3s/step - loss: 0.9146 - val_loss: 0.7179
Epoch 2/5
400/400 [==============================] - 1665s 4s/step - loss: 0.6457 - val_loss: 0.7033
Epoch 3/5
400/400 [==============================] - 1435s 4s/step - loss: 0.6185 - val_loss: 0.7096
Epoch 4/5
400/400 [==============================] - 1564s 4s/step - loss: 0.4078 - val_loss: 0.7071
Epoch 5/5
400/400 [==============================] - 1549s 4s/step - loss: 0.3839 - val_loss: 0.7152


In [9]:
modified_classifier.save('xception.h5')

In [37]:
from keras.models import load_model

In [53]:

Xception = load_model('xception.h5')

In [29]:
from keras.preprocessing.image import ImageDataGenerator

In [30]:
dataGenerator = ImageDataGenerator(rescale=1./255)
generator = dataGenerator.flow_from_directory(
        'extracted_imgs',
        shuffle=False,
        target_size=(256, 256),
        batch_size=5,
        class_mode='binary',
        subset='training')

# Predict 
# y: Deepfake, Deepfake, Deepfake, real, real 
X_test, y_test = generator.next()
image_names = generator.filenames


Found 5 images belonging to 2 classes.


In [21]:
def convertToActual(probs):
    return np.round(probs)

def computeAccuracy(pred, real):
    cnt = 0
    for i in range(len(pred)):
        if pred[i] == real[i]:
            cnt += 1
    return cnt/len(pred)*100

In [68]:
print("The order of the files are (df is DeepFake):")
for name in image_names:
        print(name)

prob_real_Xception = Xception.predict(X_test)



actual_pred_Xception = convertToActual(prob_real_Xception)
print(prob_real_Xception.shape)
print('Predicted probability of the image being real Xception:', prob_real_Xception,'\nPredicted class :', actual_pred_Xception)
print('Deepfake detection accuracy is:', computeAccuracy(actual_pred_Xception, y_test), '%')

The order of the files are (df is DeepFake):
df/elon_fake_1.png
df/elon_fake_2.png
df/obama_fake.png
real/elon_real.png
real/obama_real.png
(5, 1)
Predicted probability of the image being real Xception: [[0.48907715]
 [0.48908815]
 [0.48911962]
 [0.48907894]
 [0.48906732]] 
Predicted class : [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Deepfake detection accuracy is: 60.0 %
